In [1]:
import pandas as pd
import re
from tqdm import tqdm
import torch
import torchmetrics
import ast
import random
import transformers
from transformers import GPT2Config, GPT2Model
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
import tree
import functions
from functions import Dataset_create_embeddings_from_tokens_for_text, Clinical_Bert_Model, Dataset_create_embeddings_from_classdescripsion_for_classsentence_list
import sequence_label
import wandb
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import multiprocessing as mp
import argparse_para

/u/home/niur/venv_1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/home/niur/venv_1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
args = argparse_para.arg_parser()

In [3]:
tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease = tree.build_a_tree()
# build the tree, it is four dicts.

100%|██████████| 1391/1391 [00:00<00:00, 30567.08it/s]


In [4]:

tokenizer = transformers.BertTokenizer.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b', ignore_mismatched_sizes=True)
clinical_DS_bert = transformers.BertModel.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b')
# get the pre-trained bert model.   

Some weights of the model checkpoint at ./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print('GPU', torch.cuda.is_available())

clinical_Bert_Model = Clinical_Bert_Model(clinical_DS_bert=clinical_DS_bert)
clinical_Bert_Model = clinical_Bert_Model.to(device)

GPU True


In [6]:
# this dataframe contains the incd9 classes embeddings

df_classes_descripsion = functions.create_classes_descripsions(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
# df_classes_descripsion_embeddings has two columns: 'classes' and 'descripsion', and the length of it is all the category and subclass(end tokens not included).

class_input_ids_tensor = functions.create_class_bert_tokens(df_classes_descripsion, tokenizer)  # , class_token_type_idstensor, class_attention_mask_tensor
class_input_ids_tensor = class_input_ids_tensor.detach()
create_class_embeddings_dataset = Dataset_create_embeddings_from_classdescripsion_for_classsentence_list(class_input_ids_tensor)  # , class_token_type_idstensor, class_attention_mask_tensor
create_class_embeddings_dataloader = DataLoader(create_class_embeddings_dataset, 1, shuffle=False, drop_last=False, num_workers=8)
class_embeddings_tensor = functions.create_classes_embeddings(create_class_embeddings_dataloader, clinical_Bert_Model, device)
print('the shape of class_embedding_tensor is: ', class_embeddings_tensor.shape)
class_embeddings_dict = functions.create_class_embeddings_dict(list(df_classes_descripsion['classes']), class_embeddings_tensor)
print('the length of class embedding is(end tokens not included)',len(class_embeddings_dict))
# keys are classes and values are embeddings tensors, and the length of it is all the category and subclass(end tokens not included).
# the shape of embeddings is (n, 768)
torch.cuda.empty_cache()

100%|██████████| 15978/15978 [03:14<00:00, 82.07it/s]


the shape of class_embedding_tensor is:  torch.Size([15978, 768])
the length of class embedding is(end tokens not included) 15978


In [7]:
print(class_embeddings_tensor.device)

cuda:0


In [8]:
df_train_top500 = pd.read_csv('./data/top500_datasets/trainset_top500_new.csv').iloc[:32]
df_validation_top500 = pd.read_csv('./data/top500_datasets/validationset_top500_new.csv')

In [9]:
df_train_top500 = functions.text_cleaning(df_train_top500)
df_validation_top500 = functions.text_cleaning(df_validation_top500)
df_train_top500 = functions.convert_icdstr_to_list(df_train_top500)
df_validation_top500 = functions.convert_icdstr_to_list(df_validation_top500)
df_train_top500 = sequence_label.create_sequence_label_depth_first(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, df_train_top500)
df_validation_top500 = sequence_label.create_sequence_label_depth_first(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, df_validation_top500)
print(df_train_top500.columns)
print(df_train_top500.shape, df_validation_top500.shape)


/u/home/niur/htc_mimic3/functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub(r'\[\*\*.*?\*\*\]', '', dataframe['TEXT'].iloc[i])  # delete [**      **]
/u/home/niur/htc_mimic3/functions.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub('[0-9]{2}:[0-9]{2}', '', dataframe['TEXT'].iloc[i])  # delete 12:11
/u/home/niur/htc_mimic3/functions.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

Index(['HADM_ID', 'TEXT', 'ICD9_CODE', 'SEQUENCIAL_LABEL'], dtype='object')
(32, 4) (1468, 4)


In [10]:


input_ids_list_train = functions.create_text_tokens(df_train_top500, tokenizer) # , token_type_idstensor_train, attention_mask_tensor_train
# input_ids_list_train = input_ids_list_train.detach()
print('train token shape:', len(input_ids_list_train))
input_ids_list_validation = functions.create_text_tokens(df_validation_top500, tokenizer) # , token_type_idstensor_validation, attention_mask_tensor_validation
# input_ids_list_validation = input_ids_list_validation.detach()
print('valid token shape:', len(input_ids_list_validation))

# train_dataset_top500_create_text_embeddings = Dataset_create_embeddings_from_tokens_for_text(input_ids_tensor_train) # , token_type_idstensor_train, attention_mask_tensor_train
# validation_datasdet_top500_create_text_embeddings = Dataset_create_embeddings_from_tokens_for_text(input_ids_tensor_validation) #, token_type_idstensor_validation, attention_mask_tensor_validation
# train_dataloader_top500_create_text_embeddings = DataLoader(train_dataset_top500_create_text_embeddings, 1, shuffle=False, drop_last=False, num_workers=8)
# validation_dataloader_top500_create_text_embeddings = DataLoader(validation_datasdet_top500_create_text_embeddings, 1, shuffle=False, drop_last=False, num_workers=8)


# train_text_embeddings_tensor = functions.create_text_embeddings(train_dataloader_top500_create_text_embeddings, clinical_Bert_Model, device)
# validation_text_embeddings_tensor = functions.create_text_embeddings(validation_dataloader_top500_create_text_embeddings, clinical_Bert_Model, device)
# # this is the outputs of encoder and will be the cross attention input of decoder.
# print('train text embedding shape:', train_text_embeddings_tensor.shape)
# print('valid text embedding shape:', validation_text_embeddings_tensor.shape)

100%|██████████| 32/32 [00:00<00:00, 32.57it/s]


train token shape: 32


100%|██████████| 1468/1468 [00:46<00:00, 31.75it/s]

valid token shape: 1468


In [11]:
mask_dict = functions.create_tokens_mask_dict(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
# it s a dictionary contains the -inf and 0 mask(1d tensor) of each category and subclass.
class_label_index_dict = functions.create_class_label_index_dict(mask_dict)
# {'start':0, .......}
index_to_class_label_dict = functions.create_index_to_class_label_dict(mask_dict)
# {0:'start', .......}
print('the length of mask dict is:',len(class_label_index_dict))

the length of mask dict is: 17392


In [12]:
train_sequencial_label_list = list(df_train_top500['SEQUENCIAL_LABEL'])       # [[L], [L], [L]......]
valiation_sequencial_label_list = list(df_validation_top500['SEQUENCIAL_LABEL'])
# here the data pass into the dataset can not be tensors, because change into embeddings need the complete class embedding dict.

train_label_index_list = functions.convrt_sequencial_label_to_label_index(class_label_index_dict, train_sequencial_label_list)  # [tensor(L), tensor(L), tensor(L), tensor(L), tensor(L).......]
validation_label_index_list = functions.convrt_sequencial_label_to_label_index(class_label_index_dict, valiation_sequencial_label_list)

In [13]:
class Dataset_transformer(Dataset):
    def __init__(self, input_ids_list_train, label_index_list):
        self.input_ids_list_train = input_ids_list_train   #[(n,128), (n,128), (n,128), (n,128), (n,128), (n,128), (n,128)]
        self.label_index_list = label_index_list      # [tensor(L), tensor(L), tensor(L), tensor(L), tensor(L).......]

    def __len__(self):
        return len(self.input_ids_list_train)

    def __getitem__(self, index):
        return self.input_ids_list_train[index], self.label_index_list[index]
    
def collate_fn(batch):
    batch_input_ids_list = [item[0] for item in batch]  #[(n,128), (n,128), (n,128), (n,128), (n,128), (n,128), (n,128)]
    batch_input_ids_tensor = torch.nn.utils.rnn.pad_sequence(batch_input_ids_list, batch_first=True)  # (b,n,128)  n is the largest number of sentences in a samples batch.
    encoder_output_mask = []
    for item in batch_input_ids_list:
        single_mask = torch.tensor([0]*batch_input_ids_tensor.shape[1])  # masked is 0
        single_mask[:item.shape[0]] = 1    # not masked is 1
        encoder_output_mask.append(single_mask)
    encoder_output_mask = torch.stack(encoder_output_mask) # (b,n)
    
    
    batch_label_index = torch.nn.utils.rnn.pad_sequence([item[1] for item in batch], batch_first=True, padding_value=17391)  # (b,L) L is the largest length in this batch
    max_L = batch_label_index.shape[1]
    padding_mask = []
    for item in batch:
        single_mask = torch.tensor([0]*max_L)  # masked is 0
        true_L = item[1].shape[0]
        single_mask[:true_L] = 1    # not masked is 1
        padding_mask.append(single_mask)
    padding_mask = torch.stack(padding_mask)   # (b,L) L is the largest length in this batch   (1,0)

    
    return batch_input_ids_tensor, encoder_output_mask, batch_label_index, padding_mask     #  (b,n,128) (b,n) (b,L) (b,L)  L is the largest length in this batch 


train_dataset = Dataset_transformer(input_ids_list_train, train_label_index_list)
validation_dataset = Dataset_transformer(input_ids_list_validation, validation_label_index_list)
batch_size_transformer = args.batch_size_transformer
num_workers = args.num_workers
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size_transformer, shuffle=True, drop_last=True, num_workers=num_workers, collate_fn=collate_fn, pin_memory=True)  
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=batch_size_transformer, shuffle=False, drop_last=False, num_workers=num_workers, collate_fn=collate_fn, pin_memory=True)
decoder_num_layers = args.decoder_num_layers
transformer_dropout = args.transformer_dropout
transformer_nhead = args.transformer_nhead

In [14]:
class Transformer_Model(torch.nn.Module):
    def __init__(self):
        super(Transformer_Model, self).__init__()

        self.clinical_bert = clinical_DS_bert
        for lay in self.clinical_bert.encoder.layer[args.bert_freeze_layerindex_start:args.bert_freeze_layerindex_end+1]:
            for para in lay.parameters():
                para.requires_grad = False

        self.class_embeddings_start = torch.nn.Parameter(torch.randn(768))
        self.class_embeddings_end_start = torch.nn.Parameter(torch.randn(768))
        
        self.gpt2config = transformers.GPT2Config(n_embd=768, n_layer=decoder_num_layers, n_head=transformer_nhead, add_cross_attention=True, n_positions=6670)
        self.gpt2model = torch.nn.ModuleList([transformers.models.gpt2.modeling_gpt2.GPT2Block(self.gpt2config) for i in range(self.gpt2config.num_hidden_layers)])

        # self.decoder_layer = torch.nn.TransformerDecoderLayer(d_model=768, nhead=transformer_nhead, dropout=transformer_dropout, activation='gelu', batch_first=True)
        # self.transformer_decoder = torch.nn.TransformerDecoder(self.decoder_layer, num_layers=decoder_num_layers)  # forward padding mask(N,L)  attention causal mask(L,L)  multiheadattention  is_causal

        self.linear1 = torch.nn.Linear(768, 768)
        self.gelu = torch.nn.GELU()
        self.linear2 = torch.nn.Linear(768, 768)
        self.softmax = torch.nn.Softmax()

    def forward(self, class_embeddings_dict, mask_dict, batch_input_ids_tensor, batch_label_index, batch_padding_mask, encoder_output_mask):   #  (b,n,128), (b,L)  (b,L)  (b,n)

        class_embeddings_dict = functions.add_end_classes_embeddings(class_embeddings_dict, self.class_embeddings_start, self.class_embeddings_end_start, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, device)
        class_embedding_list = []
        for item in list(mask_dict.keys())[:-1]:
            class_embedding_list.append(class_embeddings_dict[item])
        class_embeddings_tensor = torch.stack(class_embedding_list)    # (C,768) C:17391

        label_embedding_tensor = functions.convert_sequencial_label_to_embedding_tensor(batch_label_index, class_embeddings_dict, index_to_class_label_dict)  # (b,L) --> (b,L,768)
        
        batch_text_embedding_tensor = torch.stack([self.clinical_bert(single_input_ids_tensor)['last_hidden_state'][:,0,:] for single_input_ids_tensor in batch_input_ids_tensor])   # (b,n,768)
        batch_padding_mask = batch_padding_mask[:,None,:,None]
        encoder_output_mask = encoder_output_mask[:,None,None,:]

        for i, block in enumerate(self.gpt2model):
            gpt2model_output = block(label_embedding_tensor, layer_past=None, attention_mask=batch_padding_mask,
                                encoder_hidden_states=batch_text_embedding_tensor, encoder_attention_mask=encoder_output_mask, use_cache=True)[0]#  (b,L,768)  (b,n,768) -->  (b,L,768)


        transformer_output = self.linear1(gpt2model_output)   #  (b,L,768)
        transformer_output = self.gelu(transformer_output)
        transformer_output = self.linear2(transformer_output)  #  (b,L,768)

        transformer_output = torch.nn.functional.normalize(transformer_output, dim=-1)  # (b,L,768)
        class_embeddings_tensor = torch.nn.functional.normalize(class_embeddings_tensor, dim=-1)  #  (C,768)


        score_tensor = torch.matmul(transformer_output, class_embeddings_tensor.T).to(dtype=torch.float16)  # (b,L,768), (768,C)  -->  (b,L,C)
        batch_mask_list = []
        for item in batch_label_index:  # (b,L)
            batch_mask_list.append(torch.stack([mask_dict[index_to_class_label_dict[index.item()]] for index in item]))   # (L,C)
        batch_mask_tensor = torch.stack(batch_mask_list)   # (b,L,C)
        batch_mask_tensor = batch_mask_tensor.to(device)
        logits = (score_tensor+batch_mask_tensor)   # (b,L,C)
        return logits  # (b,L,C)
    
    def forward_valid(self, class_embeddings_dict, mask_dict, class_embeddings_tensor, batch_text_embedding_tensor, batch_label_index, encoder_output_mask, past_key_values):   #  (1,n,768), (1,L)  (1,L)  (1,n)


        label_embedding_tensor = functions.convert_sequencial_label_to_embedding_tensor(batch_label_index, class_embeddings_dict, index_to_class_label_dict)  # (1,L) --> (1,L,768)
        
        past_key_values_temp = []
        encoder_output_mask = encoder_output_mask[:,None,None,:]
        if not past_key_values:
            for i, block in enumerate(self.gpt2model):
                gpt2model_output = block(label_embedding_tensor, layer_past=None, attention_mask=None,
                                encoder_hidden_states=batch_text_embedding_tensor, encoder_attention_mask=encoder_output_mask, use_cache=True)  # (1,L,768)  (1,n,768) -->  (1,L,768)

                past_key_values_temp.append(gpt2model_output[1])

        else:
            for i, (layer_past, block) in enumerate(zip(past_key_values, self.gpt2model)):
                # layer_past = tuple(past_state.to(device) for past_state in layer_past)
                gpt2model_output = block(label_embedding_tensor, layer_past=layer_past, attention_mask=None,
                                encoder_hidden_states=batch_text_embedding_tensor, encoder_attention_mask=encoder_output_mask, use_cache=True)  # (1,L,768)  (1,n,768) -->  (1,L,768)
                # print(gpt2model_output[1][0].shape)
                past_key_values_temp.append(gpt2model_output[1])

        transformer_output = self.linear1(gpt2model_output[0])   #  (1,L,768)
        transformer_output = self.gelu(transformer_output)
        transformer_output = self.linear2(transformer_output)  #  (1,L,768)
        transformer_output = torch.nn.functional.normalize(transformer_output, dim=-1)  # (1,L,768)
        class_embeddings_tensor = torch.nn.functional.normalize(class_embeddings_tensor, dim=-1)  #  (C,768)

        
        score_tensor = torch.matmul(transformer_output, class_embeddings_tensor.T).to(dtype=torch.float16)  # (1,L,768), (768,C)  -->  (1,L,C)
        batch_mask_list = []
        for item in batch_label_index:  # (b,L)
            batch_mask_list.append(torch.stack([mask_dict[index_to_class_label_dict[index.item()]] for index in item]))   # (L,C)
        batch_mask_tensor = torch.stack(batch_mask_list)   # (1,L,C)
        batch_mask_tensor = batch_mask_tensor.to(device)
        logits = (score_tensor+batch_mask_tensor)   # (1,L,C)
        return logits, past_key_values_temp  # (1,L,C)
        

In [15]:
transformer_model = Transformer_Model()
transformer_model = transformer_model.to(device)
learning_rate = args.learning_rate
criterion = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=17391)    # when compute, the size must be (N, C, L) and (N, L)
optimizer = torch.optim.AdamW(transformer_model.parameters(), lr=learning_rate, weight_decay=args.adam_weight_decay)

In [19]:
def greedy_search(class_embeddings_dict_valid, single_input_ids_tensor_valid, class_embeddings_tensor, single_encoder_output_mask):
    predicted_index_list = []
    predicted_token_list = []
    predicted_token_probabiluty_list = []
    input_token_index_list = [class_label_index_dict['start']]
    end_start_index_label = class_label_index_dict['/start']
    batch_text_embedding_tensor = torch.stack([transformer_model.clinical_bert(single_input_ids_tensor)['last_hidden_state'][:,0,:] for single_input_ids_tensor in single_input_ids_tensor_valid])   # (1,n,768)
    past_key_values = None
    while input_token_index_list[-1] != end_start_index_label:
        logits, past_key_values = transformer_model.forward_valid(class_embeddings_dict_valid, mask_dict, class_embeddings_tensor, batch_text_embedding_tensor=batch_text_embedding_tensor, batch_label_index=torch.tensor([input_token_index_list]).to(device, non_blocking=True), encoder_output_mask=single_encoder_output_mask, past_key_values=past_key_values)   # (1,n,768), (1,L) --> (1,L,C)
        predicted_index = torch.argmax(logits.squeeze(0), dim=-1)[-1].item()   # (1,L,C)-->int
        predicted_token_probabiluty_list.append(logits.squeeze(0)[-1])  # (1,L,C) -->[L,(C)]
        predicted_index_list.append(predicted_index)   # [L]
        predicted_token_list.append(index_to_class_label_dict[predicted_index])  # [L]
        input_token_index_list.append(predicted_index)   # [L]

        if len(predicted_token_list) > 114:
            break
    return predicted_token_probabiluty_list, predicted_index_list, predicted_token_list


def beam_search(class_embeddings_dict_valid, single_input_ids_tensor_valid, class_embeddings_tensor, single_encoder_output_mask, beam_size=args.beam_size):  # (1,n,128)  (1,n)
    predicted_index_list = [[] for _ in range(beam_size)]
    predicted_token_probabiluty_list = [[] for _ in range(beam_size)]
    input_token_index_tensor = torch.tensor([[class_label_index_dict['start']]]).to(device, non_blocking=True)   # (1,1)
    end_start_index_label = class_label_index_dict['/start']
    batch_text_embedding_tensor = torch.stack([transformer_model.clinical_bert(single_input_ids_tensor)['last_hidden_state'][:,0,:] for single_input_ids_tensor in single_input_ids_tensor_valid])   # (1,n,768)
    past_key_values = None
    logits, past_key_values = transformer_model.forward_valid(class_embeddings_dict_valid, mask_dict, class_embeddings_tensor, batch_text_embedding_tensor=batch_text_embedding_tensor, batch_label_index=input_token_index_tensor, encoder_output_mask=single_encoder_output_mask, past_key_values=past_key_values)   # (1,n,768), (1,1) --> (1,1,C)    ((1, n_head, 1, 768/n_head), (1, n_head, 1, 768/n_head))*n_layers
    for i in range(len(past_key_values)):
        past_key_values[i] = list(past_key_values[i])
        past_key_values[i][0] = past_key_values[i][0].repeat((beam_size, 1, 1, 1))
        past_key_values[i][1] = past_key_values[i][1].repeat((beam_size, 1, 1, 1))
        past_key_values[i] = tuple(past_key_values[i])
    top = torch.topk(logits.squeeze(), beam_size, dim=-1)
    predicted_index = top[1]  # tensor (beam)
    predicted_prob_accumu = top[0]  # tensor (beam)
    for i in range(beam_size):
        predicted_index_list[i].append(predicted_index[i])
        predicted_token_probabiluty_list[i].append(predicted_prob_accumu[i])
        # predicted_token_list[i].append(index_to_class_label_dict[predicted_index[i]])
    batch_text_embedding_tensor = batch_text_embedding_tensor.repeat((beam_size, 1, 1))    # (beam,n,768)
    single_encoder_output_mask = single_encoder_output_mask.repeat((beam_size, 1))   # (beam, n)
    input_token_index_tensor = predicted_index.reshape((beam_size,1))   # (beam,1)
    while not input_token_index_tensor[:,-1].to(device).equal(torch.tensor([end_start_index_label]*beam_size).to(device)):
        logits, past_key_values = transformer_model.forward_valid(class_embeddings_dict_valid, mask_dict, class_embeddings_tensor, batch_text_embedding_tensor=batch_text_embedding_tensor, batch_label_index=input_token_index_tensor, encoder_output_mask=single_encoder_output_mask, past_key_values=past_key_values)   # (beam,n,768), (beam,L) --> (beam,L,C)
        logits = torch.nn.Softmax(dim=-1)(logits[:,-1,:])  # (beam, C)
        predicted_prob_single = logits  # (beam, C)
        logits = logits * predicted_prob_accumu.reshape((beam_size,1))
        top = torch.topk(logits.view(-1), beam_size)  # (beam*C)-->(beam)
        predicted_index = top[1]
        predicted_prob_accumu = top[0]
        come_from = torch.div(predicted_index, 17391, rounding_mode='floor')
        predicted_index = predicted_index % 17391
        past_kv_temp = past_key_values.copy()
        predicted_index_list_temp = predicted_index_list.copy()
        predicted_token_probabiluty_list_temp = predicted_token_probabiluty_list.copy()
        for i in range(beam_size):
            predicted_index_list[i] = predicted_index_list_temp[come_from[i]] + [predicted_index[i]]
            predicted_token_probabiluty_list[i] = predicted_token_probabiluty_list_temp[come_from[i]] + [predicted_prob_single[come_from[i]][predicted_index[i]]]
            for j in range(args.decoder_num_layers):
                past_key_values[j][0][i] = past_kv_temp[j][0][come_from[i]]
                past_key_values[j][1][i] = past_kv_temp[j][1][come_from[i]]
        input_token_index_tensor = [[] for _ in range(beam_size)]
        for i in range(beam_size):
            input_token_index_tensor[i] = torch.tensor(predicted_index_list[i])
        input_token_index_tensor = torch.stack(input_token_index_tensor)
        if input_token_index_tensor.shape[1] > 114:
            break
        # print(len(predicted_token_probabiluty_list[0]), len(predicted_index_list[0]))
    predicted_index_list = predicted_index_list[0]
    cut_index = predicted_index_list.index(end_start_index_label) if end_start_index_label in predicted_index_list else len(predicted_index_list)
    predicted_index_list = predicted_index_list[:cut_index+1]
    predicted_token_list = [index_to_class_label_dict[int(i)] for i in predicted_index_list]
    predicted_token_probabiluty_list = predicted_token_probabiluty_list[0][:cut_index+1]
    return predicted_token_probabiluty_list, predicted_index_list, predicted_token_list


In [17]:
def valid(validation_dataloader):
    transformer_model.eval()
    print('Validation start...')
    with torch.no_grad():
        validation_loss = 0
        category_accuracy = 0
        first_class_accuracy = 0
        second_class_accuracy = 0
        disease_accuracy = 0
        label_f1_0_1_tensor = torch.tensor([])
        predicted_f1_0_1_tensor = torch.tensor([])
        label_AUROC_tesnor = torch.tensor([],dtype=int).to(device)
        predicted_AUROC_tensor = torch.tensor([]).to(device)
        random_outout_index = random.randint(0,1000)

        class_embeddings_dict_valid = functions.add_end_classes_embeddings(class_embeddings_dict, transformer_model.class_embeddings_start, transformer_model.class_embeddings_end_start, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, device)
        class_embedding_list = []
        for item in list(mask_dict.keys())[:-1]:
            class_embedding_list.append(class_embeddings_dict_valid[item])
        class_embeddings_tensor = torch.stack(class_embedding_list)    # (C,768) C:17391
        print(class_embeddings_tensor.shape)

        for i, (batch_input_ids_tensor, batch_encoder_output_mask, batch_label_index, batch_padding_mask) in tqdm(enumerate(validation_dataloader)):

            # torch.cuda.empty_cache()
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                batch_input_ids_tensor = batch_input_ids_tensor.to(device, non_blocking=True)
                batch_label_index = batch_label_index.to(device, non_blocking=True)
                batch_padding_mask = batch_padding_mask.to(device, non_blocking=True)
                batch_encoder_output_mask = batch_encoder_output_mask.to(device, non_blocking=True)
                # attention_mask_valid = torch.triu(torch.full((batch_padding_mask.shape[1], batch_padding_mask.shape[1]), True), diagonal=1).to(device, non_blocking=True)   # masked is True, not masked is False
                

                batch_logits = transformer_model(class_embeddings_dict_valid, mask_dict, batch_input_ids_tensor=batch_input_ids_tensor, batch_label_index=batch_label_index, batch_padding_mask=batch_padding_mask, encoder_output_mask=batch_encoder_output_mask)   # (b,n,128), (b,L) --> (b,L,C)
                batch_logits = batch_logits.permute((0,2,1))   # --> (b,C,L)
                end_start_index_label = class_label_index_dict['/start'] 
                lable_for_loss_tensor = functions.create_lable_for_loss_tensor(batch_label_index, end_start_index_label, device) # (b,L)
                validation_loss += criterion(batch_logits, lable_for_loss_tensor)

                for j in range(batch_input_ids_tensor.shape[0]):
                    single_input_ids_tensor_valid = batch_input_ids_tensor[j].unsqueeze(0)   # (1,n,128)
                    single_encoder_output_mask = batch_encoder_output_mask[j].unsqueeze(0)  # (1,n)

                    single_label_index_list = batch_label_index[j].tolist()    # [L]
                    padding_index = single_label_index_list.index(17391) if 17391 in single_label_index_list else len(single_label_index_list)
                    single_label_index_list = single_label_index_list[:padding_index]
                    single_sequencial_label = [index_to_class_label_dict[index] for index in single_label_index_list]

                    # ingle_label_index_tensor = label_index_list[j].unsqueeze(0)    # (1,L)
                    # single_text_embedding_tensor = single_text_embedding_tensor.to(device)
                    # single_label_index_tensor = single_label_index_tensor.to(device)

                    if args.generative_strategy == 'greedy_search':
                        predicted_token_probabiluty_list, predicted_index_list, predicted_token_list = greedy_search(class_embeddings_dict_valid, single_input_ids_tensor_valid, class_embeddings_tensor, single_encoder_output_mask)
                    elif args.generative_strategy == 'beam_search':
                        predicted_token_probabiluty_list, predicted_index_list, predicted_token_list = beam_search(class_embeddings_dict_valid, single_input_ids_tensor_valid, class_embeddings_tensor, single_encoder_output_mask)
                    # predicted_index_list = []
                    # predicted_token_list = []
                    # predicted_token_probabiluty_list = []
                    # input_token_index_list = [class_label_index_dict['start']]
                    # end_start_index_label = class_label_index_dict['/start']
                    # batch_text_embedding_tensor = torch.stack([transformer_model.clinical_bert(single_input_ids_tensor)['last_hidden_state'][:,0,:] for single_input_ids_tensor in single_input_ids_tensor_valid])   # (b,n,768)
                    # past_key_values = None
                    # while input_token_index_list[-1] != end_start_index_label:
                    #     logits, past_key_values = transformer_model.forward_valid(class_embeddings_dict_valid, mask_dict, class_embeddings_tensor, batch_text_embedding_tensor=batch_text_embedding_tensor, batch_label_index=torch.tensor([input_token_index_list]).to(device, non_blocking=True), encoder_output_mask=single_encoder_output_mask, past_key_values=past_key_values)   # (1,n,128), (1,L) --> (1,L,C)
                    #     predicted_index = torch.argmax(logits.squeeze(0), dim=-1)[-1].item()   # (1,L,C)-->int
                    #     predicted_token_probabiluty_list.append(logits.squeeze(0)[-1])  # (1,L,C) -->[L,(C)]
                    #     predicted_index_list.append(predicted_index)   # [L]
                    #     predicted_token_list.append(index_to_class_label_dict[predicted_index])  # [L]
                    #     input_token_index_list.append(predicted_index)   # [L]

                    #     if len(predicted_token_list) > 114:
                    #         break

                    if args.generative_strategy == 'greedy_search':
                        predicted_token_probabiluty_tensor = torch.nn.Softmax(dim=-1)(torch.stack(predicted_token_probabiluty_list))  # (L,C)
                        single_probability_AUROC_predicted, single_label_AUROC_0_1_tensor = functions.create_probability_tensor_for_AUROC(single_sequencial_label, class_label_index_dict, predicted_token_probabiluty_tensor, predicted_index_list, device)
                    elif args.generative_strategy == 'beam_search':
                        single_probability_AUROC_predicted = torch.tensor([0.0]*17391, dtype=torch.float).to(device, non_blocking=True)
                        single_probability_AUROC_predicted[predicted_index_list] = torch.tensor(predicted_token_probabiluty_list, dtype=torch.float).to(device, non_blocking=True)
                        class_list = list(class_label_index_dict.keys())
                        single_label_AUROC_0_1_tensor = torch.tensor([0]*17391)
                        for k in single_sequencial_label:
                            index = class_list.index(k)
                            single_label_AUROC_0_1_tensor[index] = 1

                    predicted_category_tokens_list, predicted_first_class_tokens_list, predicted_second_class_tokens_list, predicted_disease_tokens_list = functions.split_predicted_class_tokens(predicted_token_list, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
                    label_category_tokens_list, label_first_class_tokens_list, label_second_class_tokens_list, label_disease_tokens_list = functions.split_predicted_class_tokens(single_sequencial_label, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
                    category_accuracy += (len(set(predicted_category_tokens_list).intersection(set(label_category_tokens_list))) / max(len(predicted_category_tokens_list), len(label_category_tokens_list)))
                    first_class_accuracy += (len(set(predicted_first_class_tokens_list).intersection(set(label_first_class_tokens_list))) / max(len(predicted_first_class_tokens_list), len(label_first_class_tokens_list)))
                    second_class_accuracy += (len(set(predicted_second_class_tokens_list).intersection(set(label_second_class_tokens_list))) / max(len(predicted_second_class_tokens_list), len(label_second_class_tokens_list)))
                    disease_accuracy += (len(set(predicted_disease_tokens_list).intersection(set(label_disease_tokens_list))) / max(len(predicted_disease_tokens_list), len(label_disease_tokens_list)))

                    single_label_f1_0_1_tensor, single_predicted_f1_0_1_tensor = functions.create_0_1_tensor_for_f1(single_sequencial_label, predicted_disease_tokens_list)
                    label_f1_0_1_tensor = torch.cat([label_f1_0_1_tensor, single_label_f1_0_1_tensor])
                    predicted_f1_0_1_tensor = torch.cat([predicted_f1_0_1_tensor, single_predicted_f1_0_1_tensor])

                    
                    label_AUROC_tesnor = torch.cat([label_AUROC_tesnor, single_label_AUROC_0_1_tensor.to(device).type(torch.int)])
                    predicted_AUROC_tensor = torch.cat([predicted_AUROC_tensor, single_probability_AUROC_predicted])
                    if batch_size_transformer*i+j == random_outout_index:
                        print('label: ', single_sequencial_label)
                        print('predicted: ', predicted_token_list)



        label_f1_0_1_tensor = label_f1_0_1_tensor.reshape((df_validation_top500.shape[0], 14567))  # (N, C)
        predicted_f1_0_1_tensor = predicted_f1_0_1_tensor.reshape((df_validation_top500.shape[0], 14567))  # (N, C)
        valid_f1_micro = torchmetrics.functional.classification.multilabel_f1_score(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, threshold=0.5, average='micro', multidim_average='global')
        valid_f1_weighted = torchmetrics.functional.classification.multilabel_f1_score(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, threshold=0.5, average='weighted', multidim_average='global')


        label_AUROC_tesnor = label_AUROC_tesnor.reshape((df_validation_top500.shape[0], 17391))  # (N, C)
        predicted_AUROC_tensor = predicted_AUROC_tensor.reshape((df_validation_top500.shape[0], 17391))  # (N, C)
        valid_AUROC_micro = torchmetrics.functional.classification.multilabel_auroc(predicted_AUROC_tensor, label_AUROC_tesnor, num_labels=17391, average='micro')
        valid_AUROC_weighted = torchmetrics.functional.classification.multilabel_auroc(predicted_AUROC_tensor, label_AUROC_tesnor, num_labels=17391, average='weighted')

        valid_precision_micro = torchmetrics.functional.classification.multilabel_precision(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, multidim_average='global', average='micro')
        valid_recall_micro = torchmetrics.functional.classification.multilabel_recall(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, multidim_average='global', average='micro')

    return validation_loss/df_validation_top500.shape[0], category_accuracy/df_validation_top500.shape[0], first_class_accuracy/df_validation_top500.shape[0], second_class_accuracy/df_validation_top500.shape[0], disease_accuracy/df_validation_top500.shape[0], valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision_micro, valid_recall_micro





def train(train_dataloader, validation_dataloader, epochs=5):

    # config = { 'batch_size': batch_size_transformer, 'lr':learning_rate, 'loss':'CrossEntropyLoss', 'optim':'Adam', 
    #           'transformer_decoder_nhead':transformer_nhead, 'transformer_decoder_layers':decoder_num_layers, 'linear_layer':'768-768-Gelu-768', 'epochs':epochs}
    # wandb.init(project="htc_mimc3_personal",entity='ranniu', config=config)

    print('Training start...')
    scaler = torch.cuda.amp.GradScaler()
    iter_gradient_accumulation = args.iter_gradient_accumulation

    for epoch in range(epochs):
        print(f'Epoch: {epoch+1}')
        loss_training_epoch = 0

        transformer_model.train()
        for i, (batch_input_ids_tensor, batch_encoder_output_mask, batch_label_index, batch_padding_mask) in tqdm(enumerate(train_dataloader)):
            # torch.cuda.empty_cache()
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                loss_training_batch = 0
                batch_label_index = batch_label_index.to(device, non_blocking=True)  # (b,L)  L is the largest length in this batch 
                batch_padding_mask = batch_padding_mask.to(device, non_blocking=True)  # (b,L)  L is the largest length in this batch 
                batch_input_ids_tensor = batch_input_ids_tensor.to(device, non_blocking=True)  # (b,n,128) n is the largest number of sentenses in a sample
                batch_encoder_output_mask = batch_encoder_output_mask.to(device, non_blocking=True)  #  (b,n)  n is the largest number of sentenses in a sample
                

                # attention_mask_train = torch.triu(torch.full((batch_padding_mask.shape[1], batch_padding_mask.shape[1]), True), diagonal=1).to(device, non_blocking=True)
                # masked is True, not masked is False
                batch_logits = transformer_model(class_embeddings_dict, mask_dict, batch_input_ids_tensor=batch_input_ids_tensor, batch_label_index=batch_label_index, batch_padding_mask=batch_padding_mask, encoder_output_mask=batch_encoder_output_mask)   # (b,n,768), (b,L) --> (b,L,C)
                batch_logits = batch_logits.permute((0,2,1))    # --> (b,C,L)
                end_start_index_label = class_label_index_dict['/start'] 
                lable_for_loss_tensor = functions.create_lable_for_loss_tensor(batch_label_index, end_start_index_label, device) # (b,L)
                loss_training_batch = criterion(batch_logits, lable_for_loss_tensor)  # (b,C,L), (b,L) --> tensor

            
            scaler.scale(loss_training_batch).backward()
            if (i + 1) % iter_gradient_accumulation == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()         
            loss_training_epoch += loss_training_batch.item()

            

        
        print('the training loss in epoch',epoch+1,'is:',loss_training_epoch/df_train_top500.shape[0])
        validation_loss, category_accuracy, first_class_accuracy, second_class_accuracy, disease_accuracy, valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision_micro, valid_recall_micro= valid(validation_dataloader)
        # wandb.log({'training_loss': loss_training_epoch/df_train_top500.shape[0]})
        # wandb.log({'validation_loss': validation_loss.item()})
        # wandb.log({'valid_category_accuracy': category_accuracy})
        # wandb.log({'valid_first_class_accuracy': first_class_accuracy})
        # wandb.log({'valid_second_class_accuracy': second_class_accuracy})
        # wandb.log({'valid_disease_accuracy': disease_accuracy})
        # wandb.log({'valid_f1_micro': valid_f1_micro.item()})
        # wandb.log({'valid_f1_weighted': valid_f1_weighted.item()})
        # wandb.log({'valid_AUROC_micro': valid_AUROC_micro.item()})
        # wandb.log({'valid_AUROC_weighted': valid_AUROC_weighted.item()})
        # wandb.log({'valid_precision_micro': valid_precision_micro.item()})
        # wandb.log({'valid_recall_micro': valid_recall_micro.item()})

        print('the valid loss in epoch',epoch+1,'is:', validation_loss.item())
        print('the valid accuracy in epoch',epoch+1,'is:', category_accuracy, first_class_accuracy, second_class_accuracy, disease_accuracy)
        print('the valid f1 micro in epoch',epoch+1,'is:', valid_f1_micro.item())
        print('the valid f1 weighted in epoch',epoch+1,'is:', valid_f1_weighted.item())
        print('the valid AUROC micro in epoch',epoch+1,'is:', valid_AUROC_micro.item())
        print('the valid AUROC weighted in epoch',epoch+1,'is:', valid_AUROC_weighted.item())
        print('the valid_precision micro in epoch',epoch+1,'is:', valid_precision_micro.item())
        print('the valid_recall_micro weighted in epoch',epoch+1,'is:', valid_recall_micro.item())
    
    print('Training Finished')



In [20]:
# mp.set_start_method('spawn',force=True)
train(train_dataloader, validation_dataloader, epochs=args.epoch)

Training start...
Epoch: 1


8it [00:06,  1.29it/s]


the training loss in epoch 1 is: 104.9833459854126
Validation start...


178it [1:15:12, 25.58s/it]

label:  ['start', '390-459', '401-405', '401', '4019', '/401', '/401-405', '420-429', '427', '42731', '/427', '/420-429', '430-438', '434', '43411', '/434', '/430-438', '/390-459', '460-519', '500-508', '507', '5070', '/507', '/500-508', '/460-519', 'V', 'V10-V19', 'V12', 'V1251', '/V12', '/V10-V19', '/V']
predicted:  ['140-239', '200-208', '208', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20891', '20801', '20801', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '001-139', '00

...

the valid loss in epoch 1 is: 104.1053466796875
the valid accuracy in epoch 1 is: 0.0023540164867381854 0.000494680160892695 0.0 0.0
the valid f1 micro in epoch 1 is: 0.0
the valid f1 weighted in epoch 1 is: 0.0
the valid AUROC micro in epoch 1 is: 0.5004725456237793
the valid AUROC weighted in epoch 1 is: 0.4886965751647949
the valid_precision micro in epoch 1 is: 0.0
the valid_recall_micro weighted in epoch 1 is: 0.0
Epoch: 2


8it [00:06,  1.28it/s]

the training loss in epoch 2 is: 104.97630262374878
Validation start...



72it [30:44, 25.61s/it]


KeyboardInterrupt: 

In [ ]:
# torch.save(transformer_model, './trained_model/train02.pth')